In [1]:
%matplotlib inline

import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import defaultdict
from pathlib import Path
from tqdm.auto import tqdm

In [2]:
import sys; sys.path.append('..')
from lib.datasets import NLNOGDataset
from lib.models import HMM

In [3]:
dataset = NLNOGDataset.from_file('../dataset/NLNOG/nlnog_topology.txt', '../dataset/NLNOG/rtt_matrix.log')

### Find scenarios

In [6]:
scenarios = []

for pair in tqdm(dataset.pairs):
    direct_rtt = dataset.get_rtt(*pair)
    proxies = []

    # Find *interesting* proxy nodes
    for node in dataset.nodes:
        if node in pair:
            continue

        # Compute percentage of time the alternative path is shorter
        alt_rtt = dataset.get_rtt(pair[0], node) + dataset.get_rtt(node, pair[1])
        ratio = np.mean(alt_rtt < direct_rtt)

        if ratio > 0.1:
            proxies.append((node, ratio))
               
    scenarios.append({
        'origin': pair[0],
        'destination': pair[1],
        'proxies': proxies
    })

/home/martin/.local/share/virtualenvs/INFOCOM2020-master-aTTZR-tG/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in less
  


In [5]:
n_proxies = [len(x['proxies']) for x in scenarios]
for i, count in enumerate(np.bincount(n_proxies)):
    print('Scenarios with {} alternative path(s): {}'.format(i, count))

Scenarios with 0 alternative path(s): 165
Scenarios with 1 alternative path(s): 72
Scenarios with 2 alternative path(s): 27
Scenarios with 3 alternative path(s): 20
Scenarios with 4 alternative path(s): 23
Scenarios with 5 alternative path(s): 10
Scenarios with 6 alternative path(s): 6
Scenarios with 7 alternative path(s): 9
Scenarios with 8 alternative path(s): 7
Scenarios with 9 alternative path(s): 1
Scenarios with 10 alternative path(s): 2


### Save scenarios

In [ ]:
scenarios_dir = Path('../dataset/NLNOG/scenarios_v2')

In [ ]:
def get_scenario_dir(scenario):
    return scenarios_dir \
        .joinpath('{}_paths'.format(len(scenario['proxies'])+1)) \
        .joinpath('{}_{}'.format(scenario['origin'], scenario['destination']))

In [ ]:
for scenario in tqdm(scenarios):
    origin = scenario['origin']
    destination = scenario['destination']
    n_proxies = len(scenario['proxies'])

    # Filter on the number of proxies
    if n_proxies < 1:
        continue

    # Create the fs structure
    scenario_dir = get_scenario_dir(scenario)
    scenario_dir.mkdir(parents=True, exist_ok=True)
    
    # Save RTTs
    direct_rtt = dataset.get_rtt(origin, destination)
    direct_file = scenario_dir.joinpath('{}_{}'.format(origin, destination)).with_suffix('.csv')
    pd.DataFrame(direct_rtt).to_csv(direct_file, header=False)
    
    for proxy, _ in scenario['proxies']:
        alt_rtt = dataset.get_rtt(origin, proxy) + dataset.get_rtt(proxy, destination)
        alt_file = scenario_dir.joinpath('{}_{}_{}'.format(origin, proxy, destination)).with_suffix('.csv')
        pd.DataFrame(alt_rtt).to_csv(alt_file, header=False)

### Plot scenarios

In [ ]:
for scenario in tqdm(scenarios):
    origin = scenario['origin']
    destination = scenario['destination']
    n_proxies = len(scenario['proxies'])

    # Filter on the number of proxies
    if n_proxies < 1:
        continue
    
    plt.figure(figsize=(16,4))
    
    direct_rtt = dataset.get_rtt(origin, destination)
    plt.plot(direct_rtt, label='Direct', lw=1.0)
    
    for proxy, _ in scenario['proxies']:
        alt_rtt = dataset.get_rtt(origin, proxy) + dataset.get_rtt(proxy, destination)
        plt.plot(alt_rtt, label='via {}'.format(proxy), lw=1.0)
    
    plt.xlim(0, len(direct_rtt))
    plt.ylabel('RTT (ms)')
    plt.legend(loc='upper right')
    plt.title('{} ↔ {}'.format(origin, destination))
    
    scenario_dir = get_scenario_dir(scenario)
    scenario_name = '{}_{}'.format(origin, destination)
    plot_file = scenario_dir.joinpath(scenario_name).with_suffix('.png')

    plt.savefig(plot_file, bbox_inches='tight', dpi=200)
    plt.close()

### Plot all scenarios

In [ ]:
scenarios_by_size = defaultdict(list)
for scenario in scenarios:
    n_proxies = len(scenario['proxies'])
    scenarios_by_size[n_proxies].append(scenario)

In [ ]:
for size, scenarios in scenarios_by_size.items():
    n_scenarios = len(scenarios)
    fig, axes = plt.subplots(nrows=n_scenarios, figsize=(16, n_scenarios*3), gridspec_kw={'hspace': 0.35})
    
    for (ax, scenario) in tqdm(zip(axes, scenarios)):
        origin = scenario['origin']
        destination = scenario['destination']

        direct_rtt = dataset.get_rtt(origin, destination)
        ax.plot(direct_rtt, label='Direct', lw=1.0)

        for proxy, _ in scenario['proxies']:
            alt_rtt = dataset.get_rtt(origin, proxy) + dataset.get_rtt(proxy, destination)
            ax.plot(alt_rtt, label='via {}'.format(proxy), lw=1.0)

        ax.set_xlim(0, len(direct_rtt))
        ax.set_ylabel('RTT (ms)')
        ax.legend(loc='upper right')
        ax.set_title('{} ↔ {}'.format(origin, destination))
    
    plt.savefig('../figures/{}_paths.pdf'.format(size+1), bbox_inches='tight', dpi=200)
    plt.close()

# OLD

In [ ]:
def plot_scenario(scenario, direct_rtt, proxy_rtt):
    origin, destination, proxy = scenario
    plt.figure(figsize=(16, 4))
    plt.plot(direct_rtt, label='Direct')
    plt.plot(proxy_rtt, label='via {}'.format(proxy))
    plt.ylabel('RTT (ms)')
    plt.xlim(0, len(direct_rtt))
    plt.legend(loc='upper right')
    plt.title('{} ↔ {}'.format(origin, destination))

In [ ]:
dataset = NLNOGDataset.from_file('../dataset/NLNOG/nlnog_topology.txt', '../dataset/NLNOG/rtt_matrix.log')

In [ ]:
scenarios = [
    # origin, destination, proxy
    ('Narita', 'Boston', 'San_Francisco'),
    ('Oslo', 'Singapore', 'Warsaw'),
    ('Warsaw', 'HongKong', 'Singapore'),
    ('HongKong', 'Calgary', 'Singapore'),
    ('Boston', 'Singapore', 'Calgary'),
    ('Singapore', 'Haifa', 'Warsaw')
]

In [ ]:
scenarios_dir = Path('../dataset/NLNOG/scenarios/')

In [ ]:
for scenario in scenarios:
    origin, destination, proxy = scenario
    direct_rtt = dataset.get_rtt(origin, destination)
    proxy_rtt = dataset.get_rtt(origin, proxy) + dataset.get_rtt(proxy, destination)
    plot_scenario(scenario, direct_rtt, proxy_rtt)

In [ ]:
for scenario in scenarios:
    origin, destination, proxy = scenario
    direct_rtt = dataset.get_rtt(origin, destination)
    proxy_rtt = dataset.get_rtt(origin, proxy) + dataset.get_rtt(proxy, destination)
    
    direct_file = scenario_dir.joinpath('{}_{}'.format(origin, destination)).with_suffix('.csv')
    pd.DataFrame(direct_rtt).to_csv(direct_file, header=False)
    
    proxy_file = scenario_dir.joinpath('{}_{}_{}'.format(origin, proxy, destination)).with_suffix('.csv')
    pd.DataFrame(proxy_rtt).to_csv(proxy_file, header=False)

In [ ]:
# Plot with sample trajectories
for scenario in scenarios:
    origin, destination, proxy = scenario
    direct_rtt = dataset.get_rtt(origin, destination)
    proxy_rtt = dataset.get_rtt(origin, proxy) + dataset.get_rtt(proxy, destination)
    
    scenario_name = '{}_{}_via_{}'.format(*scenario)
    scenario_dir = scenarios_dir.joinpath(scenario_name)
    
    fig, axes = plt.subplots(nrows=3, figsize=(16, 3*4))
    
    axes[0].plot(direct_rtt, label='Direct')
    axes[0].plot(proxy_rtt, label='via {}'.format(proxy))
    axes[0].set_title('{} ↔ {}'.format(origin, destination))

    direct_model_file = scenario_dir.joinpath('{}_{}'.format(origin, destination)).with_suffix('.json')
    with open(direct_model_file) as f:
        direct_model = HMM.from_dict(json.load(f))
        
    proxy_model_file = scenario_dir.joinpath('{}_{}_{}'.format(origin, proxy, destination)).with_suffix('.json')
    with open(proxy_model_file) as f:
        proxy_model = HMM.from_dict(json.load(f))

    _, direct_obs = direct_model.sample(len(direct_rtt))
    _, proxy_obs = proxy_model.sample(len(proxy_rtt))

    axes[1].plot(direct_obs, label='Direct')
    axes[1].plot(proxy_obs, label='via {}'.format(proxy))
    axes[1].set_title('HMM models - Sample trajectory #1')
    
    _, direct_obs = direct_model.sample(len(direct_rtt))
    _, proxy_obs = proxy_model.sample(len(proxy_rtt))
    
    axes[2].plot(direct_obs, label='Direct')
    axes[2].plot(proxy_obs, label='via {}'.format(proxy))
    axes[2].set_title('HMM models - Sample trajectory #2')

    for ax in axes:
        ax.set_xlim(0, len(direct_rtt))
        ax.set_ylabel('RTT (ms)')
        ax.legend(loc='upper right')
    
    plot_file = scenario_dir.joinpath(scenario_name).with_suffix('.png')
    plt.savefig(plot_file, bbox_inches='tight', dpi=200)